In [1]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

# S3 prefix
bucket = sagemaker_session.default_bucket()
prefix = "fraud-detection-isolation-forest"

In [2]:
! mkdir data
! aws s3 cp s3://flagright-fraud-detection-data/flagright-test-transactions.json ./data

mkdir: cannot create directory ‘data’: File exists
download: s3://flagright-fraud-detection-data/flagright-test-transactions.json to data/flagright-test-transactions.json


In [3]:
WORK_DIRECTORY = "data"

train_input = sagemaker_session.upload_data(
    path="{}/{}".format(WORK_DIRECTORY, "flagright-test-transactions.json"),
    bucket=bucket,
    key_prefix="{}/{}".format(prefix, "train"),
)

In [4]:
! aws s3 cp s3://flagright-fraud-detection-data/preprocess.py ./data

download: s3://flagright-fraud-detection-data/preprocess.py to data/preprocess.py


In [5]:
from sagemaker.sklearn.estimator import SKLearn

script_path = "./data/preprocess.py"

FRAMEWORK_VERSION = "1.0-1"

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.c4.xlarge",
    sagemaker_session=sagemaker_session,
)

In [6]:
sklearn_preprocessor.fit({"train": train_input})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2023-04-08-13-11-24-123


2023-04-08 13:11:24 Starting - Starting the training job...
2023-04-08 13:11:39 Starting - Preparing the instances for training......
2023-04-08 13:12:54 Downloading - Downloading input data
2023-04-08 13:12:54 Training - Downloading the training image......
2023-04-08 13:13:50 Uploading - Uploading generated training model2023-04-08 13:13:41,262 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-04-08 13:13:41,266 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-08 13:13:41,274 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-04-08 13:13:41,460 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-08 13:13:41,471 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-08 13:13:41,484 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-08 13:13:41,492 sagemaker-tr

In [7]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, instance_type="ml.m5.xlarge", assemble_with="Line", accept="application/json"
)

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2023-04-08-13-14-37-335


In [8]:
# Preprocess training input
transformer.transform(train_input, content_type="application/json")
print("Waiting for transform job: " + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path

INFO:sagemaker:Creating transform job with name: sagemaker-scikit-learn-2023-04-08-13-14-37-865


..........................2023-04-08 13:18:45,255 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2023-04-08 13:18:45,257 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2023-04-08 13:18:45,258 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
2023-04-08 13:18:45,255 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2023-04-08 13:18:45,257 INFO - sagemaker-containers - No GPUs detected (normal if no gpus installed)
2023-04-08 13:18:45,258 INFO - sagemaker-containers - nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    se

In [9]:
! aws s3 cp s3://flagright-fraud-detection-data/train.py ./

download: s3://flagright-fraud-detection-data/train.py to ./train.py


In [10]:
from sagemaker.sklearn.estimator import SKLearn

train_script_path = "train.py"

sklearn = SKLearn(
    entry_point=train_script_path,
    framework_version=FRAMEWORK_VERSION,
    instance_type="ml.c4.xlarge",
    role=role,
    sagemaker_session=sagemaker_session,
)

In [11]:
if_train_data = sagemaker.inputs.TrainingInput(
    preprocessed_train,
    distribution="FullyReplicated",
    content_type="application/json",
    s3_data_type="S3Prefix",
)

In [12]:
sklearn.fit({"train": if_train_data})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2023-04-08-13-19-23-344


2023-04-08 13:19:23 Starting - Starting the training job...
2023-04-08 13:19:39 Starting - Preparing the instances for training......
2023-04-08 13:20:53 Downloading - Downloading input data
2023-04-08 13:20:53 Training - Downloading the training image......
2023-04-08 13:21:49 Uploading - Uploading generated training model2023-04-08 13:21:40,610 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-04-08 13:21:40,613 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-08 13:21:40,622 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-04-08 13:21:40,819 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-08 13:21:40,830 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-08 13:21:40,844 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-04-08 13:21:40,852 sagemaker-tr

In [13]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inferencee_model = sklearn_preprocessor.create_model()
sk_learn_if_model = sklearn.create_model()

model_name = "inference-pipeline-" + timestamp_prefix
endpoint_name = "inference-pipeline-ep-" + timestamp_prefix
sm_model = PipelineModel(
    name=model_name, role=role, models=[scikit_learn_inferencee_model, sk_learn_if_model]
)

sm_model.deploy(initial_instance_count=1, instance_type="ml.c4.xlarge", endpoint_name=endpoint_name)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating model with name: inference-pipeline-2023-04-08-13-22-36
INFO:sagemaker:Creating endpoint-config with name inference-pipeline-ep-2023-04-08-13-22-36
INFO:sagemaker:Creating endpoint with name inference-pipeline-ep-2023-04-08-13-22-36


------!

In [14]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer

payload = {
  "destinationAmountDetails": {
    "country": "IN",
    "transactionCurrency": "INR",
    "transactionAmount": 10132.8
  },
  "transactionState": "CREATED",
  "destinationPaymentDetails": {
    "method": "GENERIC_BANK_ACCOUNT"
  },
  "originPaymentDetails": {
    "method": "GENERIC_BANK_ACCOUNT"
  },
  "originAmountDetails": {
    "country": "IN",
    "transactionCurrency": "INR",
    "transactionAmount": 10132.8
  },
  "timestamp": {
    "$numberLong": "1662358419786"
  },
  "transactionId": "bd70fcaebc254c23b07b29fd994ba5f2",
  "originUserId": "29529892-22d3-4a74-b6f2-fbe1d5ee8b6f"
}
actual_rings = 10
predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sagemaker_session, serializer=JSONSerializer()
)

print(predictor.predict(payload))

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from container-1 with message "<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/inference-pipeline-ep-2023-04-08-13-22-36 in account 059486409817 for more information.